In [1]:
import os
from dotenv import load_dotenv

In [2]:
# load_dotenv()
# OPEN_API_KEY = os.getenv("OPEN_API_KEY")


In [3]:
load_dotenv()
GROQ_API_KEY = os.getenv("OPEN_API_KEY")


In [4]:
from langchain.document_loaders import PyPDFLoader


In [5]:
%pwd

'c:\\Users\\kumar\\Interview-question-creater\\research'

In [6]:
%cd ..


c:\Users\kumar\Interview-question-creater


In [7]:
%pwd

'c:\\Users\\kumar\\Interview-question-creater'

In [8]:
# from langchain_groq import ChatGroq

# llamaChatModel = ChatGroq(
#     model="llama3-70b-8192"
# )


In [9]:
file_path='Data/Srinivas-JDSA.pdf'
loader=PyPDFLoader(file_path)
data=loader.load()


In [10]:
data

[Document(metadata={'producer': 'Acrobat Distiller 10.1.8 (Windows)', 'creator': 'Springer', 'creationdate': '2021-12-02T06:46:51+05:30', 'author': 'A. Srinivas Reddy', 'crossmarkdomains[1]': 'springer.com', 'crossmarkdomains[2]': 'springerlink.com', 'crossmarkdomainexclusive': 'true', 'crossmarkmajorversiondate': '2010-04-23', 'keywords': 'Graph mining,Subgraph mining,Subgraph coverage patterns,Bio-informatics', 'moddate': '2021-12-02T06:48:03+05:30', 'subject': 'International Journal of Data Science and Analytics, https://doi.org/10.1007/s41060-021-00292-y', 'title': 'Mining subgraph coverage patterns from graph transactions', 'doi': '10.1007/s41060-021-00292-y', 'robots': 'noindex', 'source': 'Data/Srinivas-JDSA.pdf', 'total_pages': 17, 'page': 0, 'page_label': '1'}, page_content='International Journal of Data Science and Analytics\nhttps://doi.org/10.1007/s41060-021-00292-y\nREGULAR PAPER\nMining subgraph coverage patterns from graph transactions\nA. Srinivas Reddy 1 · P. Krishna R

In [11]:
from langchain.text_splitter import TokenTextSplitter

In [12]:
splitter_ques_gen=TokenTextSplitter(chunk_size=1000, chunk_overlap=200)


In [13]:
chunk_question_gen=splitter_ques_gen.split_documents(data)
len(chunk_question_gen)

34

In [14]:
chunk_question_gen[1]

Document(metadata={'producer': 'Acrobat Distiller 10.1.8 (Windows)', 'creator': 'Springer', 'creationdate': '2021-12-02T06:46:51+05:30', 'author': 'A. Srinivas Reddy', 'crossmarkdomains[1]': 'springer.com', 'crossmarkdomains[2]': 'springerlink.com', 'crossmarkdomainexclusive': 'true', 'crossmarkmajorversiondate': '2010-04-23', 'keywords': 'Graph mining,Subgraph mining,Subgraph coverage patterns,Bio-informatics', 'moddate': '2021-12-02T06:48:03+05:30', 'subject': 'International Journal of Data Science and Analytics, https://doi.org/10.1007/s41060-021-00292-y', 'title': 'Mining subgraph coverage patterns from graph transactions', 'doi': '10.1007/s41060-021-00292-y', 'robots': 'noindex', 'source': 'Data/Srinivas-JDSA.pdf', 'total_pages': 17, 'page': 1, 'page_label': '2'}, page_content='International Journal of Data Science and Analytics\ntions by analyzing the interaction patterns of different drug\nmolecules with the target protein.\nConsider a scenario, where we have identiﬁed a number 

In [15]:
from langchain.docstore.document import Document

In [16]:
docuemnt_ques_gen=[Document(page_content = t.page_content) for t in chunk_question_gen]

In [17]:
docuemnt_ques_gen


[Document(metadata={}, page_content='International Journal of Data Science and Analytics\nhttps://doi.org/10.1007/s41060-021-00292-y\nREGULAR PAPER\nMining subgraph coverage patterns from graph transactions\nA. Srinivas Reddy 1 · P. Krishna Reddy 1 · Anirban Mondal 2 · U. Deva Priyakumar 1\nReceived: 23 February 2021 / Accepted: 24 October 2021\n© The Author(s), under exclusive licence to Springer Nature Switzerland AG 2021\nAbstract\nPattern mining from graph transactional data (GTD) is an active area of research with applications in the domains of bioin-\nformatics, chemical informatics and social networks. Existing works address the problem of mining frequent subgraphs from\nGTD. However, the knowledge concerning the coverage aspect of a set of subgraphs is also valuable for improving the per-\nformance of several applications. In this regard, we introduce the notion of subgraph coverage patterns ( SCPs). Given a GTD,\na subgraph coverage pattern is a set of subgraphs subject to rel

In [18]:
# llm = ChatGroq(
#     model="llama3-70b-8192",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
#     api_key=GROQ_API_KEY,
    
# )

In [2]:
from langchain_groq import ChatGroq

In [19]:
##define the chat model
from langchain_groq import ChatGroq
llm_ques_gen_pipeline =ChatGroq(
    model="llama3-70b-8192",
    temperature=0.2,
    api_key=GROQ_API_KEY,
)

In [20]:
prompt_template=""" 
you are an expert at creating question based on coding materials and documents .
your goal is to prepare a coder or programmer for their exam and coding test.
You do this by asking questions about the text below:

-----------
{text}
-----------

Create questions that will prepare the coder or programmers for their tests.
Make sure not to lose any important information .

QUESTIONS:
"""

In [21]:
from langchain.prompts import PromptTemplate
prompt_ques_gen = PromptTemplate(
    template=prompt_template,
    input_variables=["text"],
)

In [22]:
refine_template = ("""

You are an expert at creating questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have teh option to refine the existing questions or create new ones.
(only if necessary) with some more contex below.

-----------
{text}
-----------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:

"""
)

In [23]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    template=refine_template,
    input_variables=["existing_answer", "text"],
)

In [24]:
from langchain.chains import load_summarize_chain

In [25]:
ques_gen_chain=load_summarize_chain(
    llm=llm_ques_gen_pipeline,
    chain_type="refine",
    verbose=True,
    question_prompt=prompt_ques_gen,
    refine_prompt=REFINE_PROMPT_QUESTIONS)

In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [36]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings


In [45]:
model="BAAI/bge-large-en"
embeddings = HuggingFaceEmbeddings(model_name=model)

c:\Users\kumar\.conda\envs\interview\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:
vector_store=FAISS.from_documents(
    documents=docuemnt_ques_gen,
    embedding=embeddings
)

In [26]:
ques=ques_gen_chain.run(docuemnt_ques_gen)
print(ques)
ques_list=ques.split('\n')

C:\Users\kumar\AppData\Local\Temp\ipykernel_24848\4145811092.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ques=ques_gen_chain.run(docuemnt_ques_gen)




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
you are an expert at creating question based on coding materials and documents .
your goal is to prepare a coder or programmer for their exam and coding test.
You do this by asking questions about the text below:

-----------
International Journal of Data Science and Analytics
https://doi.org/10.1007/s41060-021-00292-y
REGULAR PAPER
Mining subgraph coverage patterns from graph transactions
A. Srinivas Reddy 1 · P. Krishna Reddy 1 · Anirban Mondal 2 · U. Deva Priyakumar 1
Received: 23 February 2021 / Accepted: 24 October 2021
© The Author(s), under exclusive licence to Springer Nature Switzerland AG 2021
Abstract
Pattern mining from graph transactional data (GTD) is an active area of research with applications in the domains of bioin-
formatics, chemical informatics and social networks. Existing works address the problem of mining frequent subgraphs from
GTD. However, the knowled

In [27]:
ques_list

["I'd be happy to help refine the original questions and create new ones based on the provided context.",
 '',
 'Here are the refined questions:',
 '',
 '**Section 1: Graph Mining and Subgraph Coverage Patterns**',
 '',
 '1. How do graph mining techniques, such as frequent subgraph mining, facilitate the extraction of meaningful patterns and knowledge in bio-informatics and social networks, and what are the potential applications of this research in real-world scenarios, such as computer-aided drug design and optimization?',
 '',
 '2. What are the key differences between subgraph coverage patterns (SCPs) and traditional graph mining techniques, and how do user-defined constraints, such as minRF, minCS, and maxOR, influence the extraction of SCPs from graph transactional data (GTD) in terms of coverage and overlap?',
 '',
 '3. How does the SIFT framework utilize the sorted closure property of overlap ratio to efficiently prune candidate subgraph patterns and address the challenges of co

In [28]:
from langchain.chains import RetrievalQA

In [47]:

answer_genetation_chain=RetrievalQA.from_chain_type(
    llm=llm_ques_gen_pipeline,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    
)

In [ ]:
#Answer each question and save to a file
for question in ques_list:
    if question.strip():
        print(f"Question: {question}")
        answer = answer_genetation_chain.run(question)
        print(f"Answer: {answer}\n")
        print("=====================================\\n\\n")
        
        #save answer to file
        with open("answers.txt", "a") as f:
            f.write(f"Question: {question}\n")
            f.write(f"Answer: {answer}\n")
            f.write("=====================================\n\n")

Question: I'd be happy to help refine the original questions and create new ones based on the provided context.
Answer: I'm happy to help you refine your questions or create new ones based on the provided context. Please go ahead and ask your questions, and I'll do my best to assist you.

=====================================\n\n
Question: Here are the refined questions:
Answer: I'm ready to help. Please go ahead and ask your refined questions.

=====================================\n\n
Question: **Section 1: Graph Mining and Subgraph Coverage Patterns**
Answer: This section appears to be an introduction to the concept of graph mining and subgraph coverage patterns. Here's a breakdown of the content:

**Graph Mining**: Graph mining is an active area of research that involves mining knowledge from graph representations in various domains such as bioinformatics, chemical informatics, social networks, computer vision, video indexing, text retrieval, and web analysis.

**Subgraph Mining**: